# GraphCast

This colab lets you run several versions of GraphCast.

The model weights, normalization statistics, and example inputs are available on [Google Cloud Bucket](https://console.cloud.google.com/storage/browser/dm_graphcast).

A Colab runtime with TPU/GPU acceleration will substantially speed up generating predictions and computing the loss/gradients. If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".

> <p><small><small>Copyright 2023 DeepMind Technologies Limited.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Installation and Initialization


In [ ]:
# @title Pip install graphcast and dependencies

%pip install --upgrade https://github.com/deepmind/graphcast/archive/master.zip

In [ ]:
# @title Workaround for cartopy crashes

# Workaround for cartopy crashes due to the shapely installed by default in
# google colab kernel (https://github.com/anitagraser/movingpandas/issues/81):
!pip uninstall -y shapely
!pip install shapely --no-binary shapely

In [ ]:
# @title Imports

import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray


def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


In [ ]:
# @title Authenticate with Google Cloud Storage

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")

In [ ]:
# @title Plotting functions

def select(
    data: xarray.Dataset,
    variable: str,
    level: Optional[int] = None,
    max_steps: Optional[int] = None
    ) -> xarray.Dataset:
  data = data[variable]
  if "batch" in data.dims:
    data = data.isel(batch=0)
  if max_steps is not None and "time" in data.sizes and max_steps < data.sizes["time"]:
    data = data.isel(time=range(0, max_steps))
  if level is not None and "level" in data.coords:
    data = data.sel(level=level)
  return data

def scale(
    data: xarray.Dataset,
    center: Optional[float] = None,
    robust: bool = False,
    ) -> tuple[xarray.Dataset, matplotlib.colors.Normalize, str]:
  vmin = np.nanpercentile(data, (2 if robust else 0))
  vmax = np.nanpercentile(data, (98 if robust else 100))
  if center is not None:
    diff = max(vmax - center, center - vmin)
    vmin = center - diff
    vmax = center + diff
  return (data, matplotlib.colors.Normalize(vmin, vmax),
          ("RdBu_r" if center is not None else "viridis"))

def plot_data(
    data: dict[str, xarray.Dataset],
    fig_title: str,
    plot_size: float = 5,
    robust: bool = False,
    cols: int = 4
    ) -> tuple[xarray.Dataset, matplotlib.colors.Normalize, str]:

  first_data = next(iter(data.values()))[0]
  max_steps = first_data.sizes.get("time", 1)
  assert all(max_steps == d.sizes.get("time", 1) for d, _, _ in data.values())

  cols = min(cols, len(data))
  rows = math.ceil(len(data) / cols)
  figure = plt.figure(figsize=(plot_size * 2 * cols,
                               plot_size * rows))
  figure.suptitle(fig_title, fontsize=16)
  figure.subplots_adjust(wspace=0, hspace=0)
  figure.tight_layout()

  images = []
  for i, (title, (plot_data, norm, cmap)) in enumerate(data.items()):
    ax = figure.add_subplot(rows, cols, i+1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)
    im = ax.imshow(
        plot_data.isel(time=0, missing_dims="ignore"), norm=norm,
        origin="lower", cmap=cmap)
    plt.colorbar(
        mappable=im,
        ax=ax,
        orientation="vertical",
        pad=0.02,
        aspect=16,
        shrink=0.75,
        cmap=cmap,
        extend=("both" if robust else "neither"))
    images.append(im)

  def update(frame):
    if "time" in first_data.dims:
      td = datetime.timedelta(microseconds=first_data["time"][frame].item() / 1000)
      figure.suptitle(f"{fig_title}, {td}", fontsize=16)
    else:
      figure.suptitle(fig_title, fontsize=16)
    for im, (plot_data, norm, cmap) in zip(images, data.values()):
      im.set_data(plot_data.isel(time=frame, missing_dims="ignore"))

  ani = animation.FuncAnimation(
      fig=figure, func=update, frames=max_steps, interval=250)
  plt.close(figure.number)
  return HTML(ani.to_jshtml())

# Load the Data and initialize the model

## Load the model params

Choose one of the two ways of getting model params:
- **random**: You'll get random predictions, but you can change the model architecture, which may run faster or fit on your device.
- **checkpoint**: You'll get sensible predictions, but are limited to the model architecture that it was trained with, which may not fit on your device. In particular generating gradients uses a lot of memory, so you'll need at least 25GB of ram (TPUv4 or A100).

Checkpoints vary across a few axes:
- The mesh size specifies the internal graph representation of the earth. Smaller meshes will run faster but will have worse outputs. The mesh size does not affect the number of parameters of the model.
- The resolution and number of pressure levels must match the data. Lower resolution and fewer levels will run a bit faster. Data resolution only affects the encoder/decoder.
- All our models predict precipitation. However, ERA5 includes precipitation, while HRES does not. Our models marked as "ERA5" take precipitation as input and expect ERA5 data as input, while model marked "ERA5-HRES" do not take precipitation as input and are specifically trained to take HRES-fc0 as input (see the data section below).

We provide three pre-trained models.
1. `GraphCast`, the high-resolution model used in the GraphCast paper (0.25 degree resolution, 37 pressure levels), trained on ERA5 data from 1979 to 2017,

2. `GraphCast_small`, a smaller, low-resolution version of GraphCast (1 degree resolution, 13 pressure levels, and a smaller mesh), trained on ERA5 data from 1979 to 2015, useful to run a model with lower memory and compute constraints,

3. `GraphCast_operational`, a high-resolution model (0.25 degree resolution, 13 pressure levels) pre-trained on ERA5 data from 1979 to 2017 and fine-tuned on HRES data from 2016 to 2021. This model can be initialized from HRES data (does not require precipitation inputs).


In [ ]:
# @title Load the model


model = "GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz"
with gcs_bucket.blob(f"params/{model}").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)
params = ckpt.params
state = {}

model_config = ckpt.model_config
task_config = ckpt.task_config
print("Model description:\n", ckpt.description, "\n")
print("Model license:\n", ckpt.license, "\n")

model_config

## Load the example data

Several example datasets are available, varying across a few axes:
- **Source**: fake, era5, hres
- **Resolution**: 0.25deg, 1deg, 6deg
- **Levels**: 13, 37
- **Steps**: How many timesteps are included

Not all combinations are available.
- Higher resolution is only available for fewer steps due to the memory requirements of loading them.
- HRES is only available in 0.25 deg, with 13 pressure levels.

The data resolution must match the model that is loaded.

Some transformations were done from the base datasets:
- We accumulated precipitation over 6 hours instead of the default 1 hour.
- For HRES data, each time step corresponds to the HRES forecast at leadtime 0, essentially providing an "initialisation" from HRES. See HRES-fc0 in the GraphCast paper for further description. Note that a 6h accumulation of precipitation is not available from HRES, so our model taking HRES inputs does not depend on precipitation. However, because our models predict precipitation, we include the ERA5 precipitation in the example data so it can serve as an illustrative example of ground truth.
- We include ERA5 `toa_incident_solar_radiation` in the data. Our model uses the radiation at -6h, 0h and +6h as a forcing term for each 1-step prediction. If the radiation is missing from the data (e.g. in an operational setting), it will be computed using a custom implementation that produces values similar to those in ERA5.

In [ ]:
# @title Get and filter the list of available example datasets

dataset_file_options = [
    name for blob in gcs_bucket.list_blobs(prefix="dataset/")
    if (name := blob.name.removeprefix("dataset/"))
    ]  # Drop empty string.

def data_valid_for_model(
    file_name: str, model_config: graphcast.ModelConfig, task_config: graphcast.TaskConfig):
  file_parts = parse_file_parts(file_name.removesuffix(".nc"))
  return (
      model_config.resolution in (0, float(file_parts["res"])) and
      len(task_config.pressure_levels) == int(file_parts["levels"]) and
      (
          ("total_precipitation_6hr" in task_config.input_variables and
           file_parts["source"] in ("era5", "fake")) or
          ("total_precipitation_6hr" not in task_config.input_variables and
           file_parts["source"] in ("hres", "fake"))
      )
  )


dataset_file = widgets.Dropdown(
    options=[
        (", ".join([f"{k}: {v}" for k, v in parse_file_parts(option.removesuffix(".nc")).items()]), option)
        for option in dataset_file_options
        #if data_valid_for_model(option, model_config, task_config)
    ],
    description="Dataset file:",
    layout={"width": "max-content"})
widgets.VBox([
    dataset_file,
    widgets.Label(value="Run the next cell to load the dataset. Rerunning this cell clears your selection and refilters the datasets that match your model.")
])

In [ ]:
low_res_file_name = "source-era5_date-2022-01-01_res-1.0_levels-13_steps-01.nc"
high_res_file_name = "source-era5_date-2022-01-01_res-0.25_levels-13_steps-01.nc"

In [ ]:
# @title Load weather data

def load_data(file_name):
    with gcs_bucket.blob(f"dataset/{file_name}").open("rb") as f:
        example_batch = xarray.load_dataset(f).compute()

    assert example_batch.dims["time"] >= 3  # 2 for input, >=1 for targets

    return example_batch

low_res_batch = load_data(low_res_file_name)
high_res_batch = load_data(high_res_file_name)


In [ ]:
# @title Choose training and eval data to extract
train_steps = widgets.IntSlider(
    value=1, min=1, max=low_res_batch.sizes["time"]-2, description="Train steps")
eval_steps = widgets.IntSlider(
    value=low_res_batch.sizes["time"]-2, min=1, max=low_res_batch.sizes["time"]-2, description="Eval steps")

widgets.VBox([
    train_steps,
    eval_steps,
    widgets.Label(value="Run the next cell to extract the data. Rerunning this cell clears your selection.")
])

In [ ]:
# @title Extract training and eval data

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    low_res_batch, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    low_res_batch, target_lead_times=slice("6h", f"{eval_steps.value*6}h"),
    **dataclasses.asdict(task_config))

print("All Examples:  ", low_res_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)


In [ ]:
# @title Load normalization data

with gcs_bucket.blob("stats/diffs_stddev_by_level.nc").open("rb") as f:
  diffs_stddev_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob("stats/mean_by_level.nc").open("rb") as f:
  mean_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob("stats/stddev_by_level.nc").open("rb") as f:
  stddev_by_level = xarray.load_dataset(f).compute()

In [ ]:
# @title Build jitted functions, and possibly initialize random weights

def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor


@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)


@hk.transform_with_state
def loss_fn(model_config, task_config, inputs, targets, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  loss, diagnostics = predictor.loss(inputs, targets, forcings)
  return xarray_tree.map_structure(
      lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
      (loss, diagnostics))

def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is requiredy by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
  params, state = init_jitted(
      rng=jax.random.PRNGKey(0),
      inputs=train_inputs,
      targets_template=train_targets,
      forcings=train_forcings)

loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))

# Run the model

Note that the cell below may take a while (possibly minutes) to run the first time you execute them, because this will include the time it takes for the code to compile. The second time running will be significantly faster.

This use the python loop to iterate over prediction steps, where the 1-step prediction is jitted. This has lower memory requirements than the training steps below, and should enable making prediction with the small GraphCast model on 1 deg resolution data for 4 steps.

In [ ]:
# @title Autoregressive rollout (loop in python)

assert model_config.resolution in (0, 360. / eval_inputs.sizes["lon"]), (
  "Model resolution doesn't match the data resolution. You likely want to "
  "re-filter the dataset list, and download the correct data.")

print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions

# Super Resolution on top of Coarse Graphcast

### Plan:
As in https://arxiv.org/html/2409.11502v1#S1, we will treat graphcast as a blackbox and train on top of it's predictions to learn a super-resolution function that can improve a more coarse graphcasts estimations.

We will focus on wind variables as that is what Technosylva appears to be focused on.

### General workflow:
0. Load era5 data at .25 res for our variables of interest
1. Generate coarse graphcast (1 degree) predictions and select variables of interest
2. Ensure that we have the coarse predictions aligned to our ground truth data
2. Construct a held out test set for final estimation of performance
3. Model exploration for super-resolution model
4. Test selected models + non-deep learning methods on held out test set
5. Report findings along with investigation into further ways to improve this

In [ ]:
# @title Step 2: ensuring alignment between low and high res data
predictions

In [ ]:
high_res_batch

In [ ]:
# @title Visualize the difference in resolution
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

def visualize_patch(pred_ds, era5_ds,
                   lat_range=(-20, 20),
                   lon_range=(0, 40),
                   timestamp=None,
                   cmap='coolwarm',
                   variable_name='10m_u_component_of_wind',
                   figsize=(15, 6)):
    """
    Visualize a geographical patch from both prediction and ERA5 datasets.

    Args:
        pred_ds: xarray Dataset containing predictions
        era5_ds: xarray Dataset containing ERA5 data
        lat_range: Tuple of (min_lat, max_lat)
        lon_range: Tuple of (min_lon, max_lon)
        timestamp: Specific timestamp to plot (if None, uses first available)
        cmap: Colormap to use
        figsize: Figure size
    """
    # Select the region of interest
    pred_patch = pred_ds.sel(
        lat=slice(lat_range[0], lat_range[1]),
        lon=slice(lon_range[0], lon_range[1]),
        batch=0
    )

    era5_patch = era5_ds.sel(
        lat=slice(lat_range[0], lat_range[1]),
        lon=slice(lon_range[0], lon_range[1]),
        batch=0
    )

    # If timestamp not specified, use the first available
    if timestamp is None:
        if 'datetime' in pred_ds.coords:
            timestamp = pred_ds.datetime.values[0]
        else:
            timestamp = pred_ds.time.values[0]

    # Extract temperature data
    pred_temp = pred_patch[variable_name].sel(time=timestamp)
    era5_temp = era5_patch[variable_name].sel(time=timestamp)

    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2,
                                  figsize=figsize,
                                  subplot_kw={'projection': ccrs.PlateCarree()})

    # Find common temperature range for consistent coloring
    vmin = min(pred_temp.min(), era5_temp.min())
    vmax = max(pred_temp.max(), era5_temp.max())

    # Plot prediction data
    im1 = ax1.pcolormesh(pred_temp.lon, pred_temp.lat, pred_temp,
                        cmap=cmap, vmin=vmin, vmax=vmax)
    ax1.coastlines()
    ax1.gridlines(draw_labels=True)
    ax1.set_title('Prediction (Low Resolution)')

    # Plot ERA5 data
    im2 = ax2.pcolormesh(era5_temp.lon, era5_temp.lat, era5_temp,
                        cmap=cmap, vmin=vmin, vmax=vmax)
    ax2.coastlines()
    ax2.gridlines(draw_labels=True)
    ax2.set_title('ERA5 (High Resolution)')

    # Add colorbar
    plt.colorbar(im1, ax=ax1, label=variable_name)
    plt.colorbar(im2, ax=ax2, label=variable_name)

    # Add timestamp as overall title
    plt.suptitle(f'{variable_name} Comparison at {timestamp}')

    plt.tight_layout()
    return fig


fig = visualize_patch(predictions, high_res_batch,
                     lat_range=(25, 40),
                     lon_range=(110, 130))
plt.show()



## Generate lat/long pairs that will define our train/test set.
Basic idea is to create a checkerboard over the globe and sample that to construct our train/test set. If a datapoint is in a test square, then it will be in our test set


In [ ]:
import random
def get_test_lat_long_pairs(stepsize=10):
    integer_longs = np.arange(0, 360, stepsize)
    integer_lats = np.arange(-90, 90, stepsize)

    # Construct a grid
    grid = [(lat, lon) for lat in integer_lats for lon in integer_longs]

    # Sample 20% of the grid
    test_grid = random.sample(grid, int(0.2 * len(grid)))

    train_grid = [item for item in grid if item not in test_grid]

    return train_grid, test_grid

stepsize=10
test_grid, train_grid = get_test_lat_long_pairs()


In [ ]:
high_res_train = []
high_res_test = []
predictions_train = []
predictions_test = []

for test_lat,test_long in test_grid:
    high_res_test.append(high_res_batch.sel(lat=[test_lat, test_lat+stepsize], lon=[test_long, test_long+stepsize], method='nearest'))
    predictions_test.append(predictions.sel(lat=[test_lat, test_lat+stepsize], lon=[test_long, test_long+stepsize], method='nearest'))

for train_lat,train_long in train_grid:
    high_res_train.append(high_res_batch.sel(lat=[train_lat, train_lat+stepsize], lon=[train_long, train_long+stepsize], method='nearest'))
    predictions_train.append(predictions.sel(lat=[train_lat, train_lat+stepsize], lon=[train_long, train_long+stepsize], method='nearest'))

high_res_train = xarray.concat(high_res_train, dim='batch')
high_res_test = xarray.concat(high_res_test, dim='batch')
predictions_train = xarray.concat(predictions_train, dim='batch')
predictions_test = xarray.concat(predictions_test, dim='batch')



Construct pairs of input/output images by centering on lat/long


In [ ]:
# prompt: Pair up prediction and high res images by going to the center of a 5x5 degree grid that's in our training grid and finding the prediction and high res image that is centered at this

def pair_predictions_highres(predictions, high_res_batch, lat_center, lon_center, grid_size_degrees=5):
  """Pairs prediction and high-res images centered around a given lat/lon.

  Args:
    predictions: xarray Dataset containing predictions.
    high_res_batch: xarray Dataset containing high-resolution images.
    lat_center: Latitude of the center of the grid.
    lon_center: Longitude of the center of the grid.
    grid_size_degrees: Size of the grid in degrees (e.g., 5 for a 5x5 degree grid).

  Returns:
    A tuple containing the centered prediction and high-resolution image, or None
    if the center point is outside the available data.
  """
  half_grid_size = grid_size_degrees / 2.0
  lat_min = lat_center - half_grid_size
  lat_max = lat_center + half_grid_size
  lon_min = lon_center - half_grid_size
  lon_max = lon_center + half_grid_size

  try:
    prediction_patch = predictions.sel(
        lat=[lat_min,lat_max], lon=[lon_min, lon_max], method='nearest')
    high_res_patch = high_res_batch.sel(
        lat=[lat_min,lat_max], lon=[lon_min, lon_max], method='nearest')
    return prediction_patch, high_res_patch
  except KeyError:
    return None  # Center point outside data range


# Example usage:
# Assuming you have a list of lat/lon pairs you want to center on
lat_lon_pairs = [(30, 10), (45, -70), (-15, 120)]  # Replace with your actual list

for lat, lon in lat_lon_pairs:
  paired_data = pair_predictions_highres(predictions, high_res_batch, lat, lon)
  if paired_data:
    prediction_patch, high_res_patch = paired_data
    # Now you can use prediction_patch and high_res_patch for your further analysis
    print(f"Paired data found for center ({lat}, {lon})")
    # ... your processing logic ...
  else:
    print(f"No paired data found for center ({lat}, {lon})")

Train U-Net to from our low res graphcast preds to our high res era5 dataset

## Left to do
- train our models
- visualize the results
- discuss improvements